In [ ]:
import sys
import os
import gc
import collections

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np
import xarray as xr
import geopandas as gpd
import rioxarray
from xeofs.xarray import EOF, Rotator, ROCK_PCA

import statsmodels.api as sm
import scipy
from sklearn.metrics import mean_squared_error
from itertools import product
import pyproj
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd
import cmocean

# NOTE: Local xeofs to be submitted as pull-request (addition of xeofs.model.reconstruct_randomized_X method)

In [ ]:
# Define local plotting parameters
#sns.set_theme(style="whitegrid")
#sns.set_theme(style="ticks")
sns.set_theme(style="white")
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
plt.rcParams.update({'font.size': 15})
plt.rc('font', family='sans-serif') 
plt.rc('font', serif='Helvetica Neue')
plt.rc('text', usetex=True)

## Define directories

In [ ]:
# To update following with relative repository paths once data and code is up on Zenodo
# Current version uses the project template on Github.

# Define project repo path
inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'

# DATA FILE PATHS
# Data containing regridded flux and SSH for 150 years
regriddedFluxSSH_filepath = 'data/interim/RegriddedFluxSSH.nc'
# File contains all defined ice shelves
iceShelvesShape_filepath = 'data/interim/iceShelves.geojson'

# Relative directory paths for Data and Figures
figures_folderpath = 'reports/figures/'
interim_data_folder = 'data/interim/'
processed_data_folder = 'data/processed/'
flux_dedrafted_data_path = 'dedrafted_flux_IMBIE/'
randomized_realizations_path = 'randomized_realizations/'
flux_dedrafted_iceshelves_data_path = 'iceshelves_dedrafted_flux/'
reconstructions_neofs_path = 'reconstructions_neofs/'
cise_file_path = 'cise_data/'
std_file_path = 'standardized_rec_data/'


pca_1 = '1_pca/'
pca_2 = '2_pca_normalized/'
pca_3 = '3_pca_normalized_nmodes/'
pca_4 = '4_lowpass_ pca_normalized/'

## Input data files and catchment masks for ice shelves

### Catchment boundary masks

In [ ]:
# Catchment boundary masks for Antarctica, taken from ice shelf definitions in MPAS-Dev/geometric-features
# Source: https://github.com/MPAS-Dev/geometric_features/tree/main/geometric_data
# These have been combined into one file with 133 defined regions (polygons and multipolygons), 
# readable via the Geopandas package

# Read geoJSON region feature file as GeoDataFrame
iceshelvesmask = gpd.read_file(inDirName + iceShelvesShape_filepath)

# Convert to south polar stereographic projection
#icems = iceshelvesmask.to_crs({'init': 'epsg:3031'}); # This has been deprecated
icems = iceshelvesmask.to_crs('epsg:3031');
crs = ccrs.SouthPolarStereo();

### Plot catchment boundaries

In [ ]:
# Plot boundaries

plt.figure(figsize=(5,5))
ax1 = plt.subplot(121,projection=ccrs.SouthPolarStereo())
ax1.gridlines(color='whitesmoke',zorder=4)
icems[34:133].plot(ax=ax1,color='antiquewhite', linewidth=0,zorder=1)
icems[34:133].boundary.plot(ax=ax1,color='r', linewidth=0.2,zorder=3)
#icems[34:133].boundary.plot(ax=ax1,linewidth=0.25,color='lightgray',zorder=4)
#ax1.coastlines(resolution='10m', zorder=6,linewidth=0.75)
ax1.patch.set_facecolor(color='lightsteelblue')
#ax1.add_feature(cartopy.feature.LAND, color='ghostwhite')
ax1.add_feature(cartopy.feature.LAND, color='ghostwhite', zorder=2)
plt.title('Catchment Boundaries');

### Ocean forcing data

In [ ]:
# Original forcing data: raw MPAS-O output, mapped to the 10km resolution grid
# flux is freshwater flux
# ssh is sea surface height, used here as a proxy for ice draft / depth.

# ssh is used to plot the scatterplot of flux vs. draft for different ice shelves 
# and as input for the linear regression used in "dedrafting" the dataset

regridded_data = xr.open_dataset(inDirName+regriddedFluxSSH_filepath)
flux = regridded_data.timeMonthly_avg_landIceFreshwaterFlux
ssh = regridded_data.timeMonthly_avg_ssh

# Pre-processed data: detrended, deseasonalized, dedrafted
flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux

# Dedrafted flux
flux_dedrafted = xr.open_dataset(inDirName+
                                 interim_data_folder+
                                 flux_dedrafted_iceshelves_data_path+'iceshelves_dedrafted_total.nc')
flux_dedrafted = flux_dedrafted.timeMonthly_avg_landIceFreshwaterFlux

# Flux datapoints extracted for individual ice shelves, used for the scatter plots
catchments_scatter = np.load(inDirName+interim_data_folder+"catchments_scatter.npy")
catchments_scatter_xr = xr.DataArray(catchments_scatter,dims={'basin','x','y'})

# Add reconstructed datasets - seed random


# Catchment boundary masks for Antarctica, taken from ice shelf definitions in MPAS-Dev/geometric-features
# Source: https://github.com/MPAS-Dev/geometric_features/tree/main/geometric_data
# These have been combined into one file with 133 defined regions (polygons and multipolygons), 
# readable via the Geopandas package

# Read geoJSON region feature file as GeoDataFrame
iceshelvesmask = gpd.read_file(inDirName + iceShelvesShape_filepath)

# Convert to south polar stereographic projection
#icems = iceshelvesmask.to_crs({'init': 'epsg:3031'}); # This has been deprecated
icems = iceshelvesmask.to_crs('epsg:3031');
crs = ccrs.SouthPolarStereo();

In [ ]:
flux_filtered = xr.open_dataset(inDirName+processed_data_folder+pca_4+'flux_filtered_24')
flux_filtered = flux_filtered.__xarray_dataarray_variable__

### Convert Freshwater Flux [$kg. m^{-2} s^{-1}$] to Melt Rate [$m.yr^{-1}$]
Assumes density of freshwater $\rho_{fw}$ [$kg/m^3$] = 1000

In [ ]:
sec_per_year = 365*24*60*60
rho_fw = 1000

In [ ]:
catchments_scatter = catchments_scatter*sec_per_year/rho_fw

### Plot sample data for specific ice shelves (melt rate draft dependence)

In [ ]:
# Figure: Melt rate draft dependence for select catchments
# The following indices are taken from 'catchment_scatter', defined as
# Index of catchments in catchment_scatter = Index of catchments in icems - 33

# Sample ice shelves chosen:
# Amery = 1
# George VI = 27
# Getz = 28
# Pine Island = 61
# Ronne = 70
# Totten = 83
# Wilkins = 94
# Western Ross = 71
# Filchner = 22
# Thwaites = 81
# Eastern_Ross = 72

# Full list of ice shelves and indices can be obtained by printing icems

catchments = np.array([1,27,28,61,70,83,94,71,22])

#fig, ax = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})
fig, axs = plt.subplots(2,3, figsize=[15, 10])
axs=axs.ravel()

for i in range(6):
    axs[i].scatter(catchments_scatter[catchments[i]],ssh[1],color='r',s=4)
    #axs[i].set_xlabel('Melt Flux (kg / m2 / s)')
    #axs[i].set_ylabel('Depth (m)')
    axs[i].set_ylim(-1000,0) # Specified for consistency across plots, some catchments are not as deep as others
    # Uncomment to set xlim values for easy comparison of forcing magnitudes (varies across ice shelves)
    #axs[i].set_xlim(-0.5e-6,2e-5)
    axs[i].set_title(icems.name[33+catchments[i]])
    # axs[i].inset_axes([], projection=crs) does not seem to work, hence the workaround below:
    # Refer here: https://github.com/matplotlib/matplotlib/pull/22608 for future revisions
    axins = inset_axes(axs[i],width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='r',linewidth=0.4)
    axins.set_axis_off();

axs[0].set_ylabel('Depth (m)'); # Ice Draft?
axs[3].set_ylabel('Depth (m)');
#axs[6].set_ylabel('Depth (m)');

axs[3].set_xlabel('Melt Rate ($m/yr$)');
axs[4].set_xlabel('Melt Rate ($m/yr$)');
axs[5].set_xlabel('Melt Rate ($m/yr$)');

# TODO: Convert to m SLE units for flux

# Data Pre-Processing

For time series plots and comparisons, the sum of flux across the spatial domain (or specific ice shelves) is used as a proxy when required. The mean can also be used here, without any loss of information, as the idea is only to compare trendlines.
#### TODO: Verify if this is indeed true

However, this might not be possible once the data is normalized, as the magnitude of data is removed at that stage.

In [ ]:
#flux_ts = flux.sum(['x','y'])
#flux_dedrafted_ts = flux_dedrafted.sum(['x','y'])

# Processed data time series
flux_clean_ts = flux_clean.sum(['x','y'])

In [ ]:
# Remove temporal mean
# Is this necessary?
flux_demeaned = flux - flux.mean('time')
flux_demeaned_ts = flux_demeaned.sum(['x','y'])

## Remove seasonality

In [ ]:
# Deseasonalize
# Remove climatologies to isolate anomalies, i.e., internal variability
flux_month = flux_dedrafted.groupby("time.month")
flux_clm = flux_month.mean("time") # Climatologies
flux_anm = flux_month - flux_clm # Deseasonalized anomalies

In [ ]:
flux_clm_ts = flux_clm.sum(['x','y'])
flux_anm_ts = flux_anm.sum(['x','y'])

In [ ]:
plt.figure(figsize=(25,6))
plt.plot(flux_ts[300:])
plt.plot(flux_dedrafted_ts[300:])
plt.plot(flux_dedrafted_ts - flux_clean_ts)
#plt.plot(flux_clean_ts)
plt.plot(flux_anm_ts[300:])

In [ ]:
# Sanity check, as "flux_clean.nc" was created earlier. The difference here is purely numerical.

plt.figure(figsize=(25,6))
plt.plot(flux_clean_ts - flux_anm_ts[300:])

### Sparse PCA Implementation

In [ ]:
np.min

In [ ]:
flux_clean_normalized.dropna(dim='time',how='all')

In [ ]:
flux_clean_nna = flux_clean_normalized.fillna(0)
flux_clean_nna_np = flux_clean_nna.to_numpy()
flux_clean_nna_np_reshape = flux_clean_nna_np.reshape(1500,601*601)

In [ ]:
%%time
transformer = SparsePCA(n_components = 100)
transformer.fit(flux_clean_nna_np_reshape)

In [ ]:
pcs = transformer.transform(flux_clean_nna_np_reshape)
pcs.shape

# EOF/PCA Decomposition

### Data Normalization

In [ ]:
flux_clean_tmean = flux_clean.mean('time')
flux_clean_tstd = flux_clean.std('time')
flux_clean_demeaned = flux_clean - flux_clean_tmean
flux_clean_normalized = flux_clean_demeaned/flux_clean_tstd
# Why does this introduce a non-zero change? (Artefact of flux variable dimensions?)

In [ ]:
flux_filtered_tmean = flux_filtered.mean('time')
flux_filtered_tstd = flux_filtered.std('time')
flux_filtered_demeaned = flux_filtered - flux_filtered_tmean
flux_filtered_normalized = flux_filtered_demeaned/flux_filtered_tstd
# Why does this introduce a non-zero change? (Artefact of flux variable dimensions?)


Time series of the normalized data is calculated below only for illustrative purposes. It would not make sense to compare the normalized time series to other time series (which use the spatial sum or mean).

In [ ]:
# flux_clean_normalized_ts = flux_clean_normalized.mean(['x','y'])

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(flux_clean_normalized_ts,color = '#44546A', linewidth=2)
plt.title('Anomalies / Internal Variability Component (Normalized)');
plt.ylabel('Standard Deviation Units');
plt.xlabel('Time (months)');

## Spatial Decomposition: EOF Analysis

EOF decomposition is done using a modified version of the `xeofs` package.
Refer [nicrie/xeofs](https://github.com/nicrie/xeofs) on Github for documentation.

#### TODO: Upload modification of `xeofs` online, or rewrite Xarray wrapper for the scikit-learn PCA to encompass full workflow

In [ ]:
%%time
norm_model = EOF(flux_clean_normalized)
norm_model.solve()
norm_eofs = norm_model.eofs()
norm_pcs = norm_model.pcs()
norm_nmodes = norm_model.n_modes
norm_varexpl = norm_model.explained_variance_ratio()
norm_pcs_eig = norm_model.pcs(1)
norm_eofs_eig = norm_model.eofs(1)
norm_eofs_sng = norm_model.eofs(2)
norm_pcs_sng = norm_model.pcs(2)
norm_varexpl_values = norm_model.explained_variance()

In [ ]:
norm_sng = norm_model.singular_values()

In [ ]:
np.sqrt(norm_sng).plot()
np.sqrt(norm_varexpl_values).plot()

In [ ]:
%%time
model = EOF(flux_clean)
model.solve()
eofs = model.eofs()
pcs = model.pcs()
nmodes = model.n_modes
varexpl = model.explained_variance_ratio()
pcs_eig = model.pcs(1)


In [ ]:
%%time

n_rot=50 # Change this parameter
rot_var = Rotator(model, n_rot=n_rot, power=1)
rot_eofs = rot_var.eofs()
rot_pcs = rot_var.pcs()
rot_varexpl = rot_var.explained_variance_ratio()

In [ ]:
%%time

n_rot=50 # Change this parameter
rot_norm_var = Rotator(norm_model, n_rot=n_rot, power=1)
rot_norm_eofs = rot_norm_var.eofs()
rot_norm_pcs = rot_norm_var.pcs()
rot_norm_varexpl = rot_norm_var.explained_variance_ratio()

In [ ]:
%%time
filt_model = EOF(flux_filtered_normalized)
filt_model.solve()
filt_eofs = filt_model.eofs()
filt_pcs = filt_model.pcs()
filt_nmodes = filt_model.n_modes
filt_varexpl = filt_model.explained_variance_ratio()
filt_pcs_eig = filt_model.pcs(1)
filt_pcs_sng = filt_model.pcs(2)

### LOAD EOFs, PCs for plotting

In [ ]:
eofs = xr.open_dataset(inDirName+processed_data_folder+pca_1+'eofs_eig.nc')
pcs = xr.open_dataset(inDirName+processed_data_folder+pca_1+'pcs_eig.nc')
eofs = eofs.EOFs
pcs = pcs.PCs


norm_eofs = xr.open_dataset(inDirName+processed_data_folder+pca_2+'norm_eofs_eig.nc')
norm_pcs = xr.open_dataset(inDirName+processed_data_folder+pca_2+'norm_pcs_eig.nc')
norm_eofs = norm_eofs.EOFs
norm_pcs = norm_pcs.PCs

In [ ]:
import matplotlib as mpl
cmap = mpl.colormaps['inferno'](np.linspace(0,1,100))

plt.figure(figsize=(15,8))
for i in range(100):
    plt.psd(norm_pcs[:,i],noverlap=8,NFFT=256,color=cmap[i]);
    #pcs[:150,i].plot(lw=2.5, label=i, color=cmap[i])
#plt.legend()

### Plot relative power of first m modes from the PCA decomposition

In [ ]:
# Figure: Relative power of m EOFs
m = 75
sns.set_theme(style="white")
plt.figure(figsize=(15,8))
#plt.bar(range(m),varexpl[:m]*100, color='black')
plt.plot(range(m),varexpl[:m]*100, 'k*-', 
         label='Actual Data: {:.2f}\%'.format(varexpl[:m].sum().values*100))# color='black')
plt.plot(range(m),norm_varexpl[:m]*100, 'rh-', 
         label='Normalized Data: {:.2f}\%'.format(norm_varexpl[:m].sum().values*100))#color='red')
plt.plot(range(m),filt_varexpl[:m]*100, 'g.-', 
         label='Filtered \& Normalized Data: {:.2f}\%'.format(filt_varexpl[:m].sum().values*100))#color='red')
#plt.plot(range(m),rot_varexpl[:m]*100, '.--', 
#         label='Rotated EOF (Actual Data): {:.2f}\%'.format(rot_varexpl[:m].sum().values*100))# color='black')
#plt.plot(range(m),rot_norm_varexpl[:m]*100, '.--', 
#         label='Normalized Data (Rotated EOF): {:.2f}\%'.format(rot_norm_varexpl[:m].sum().values*100))# color='black')
plt.ylabel('Relative Power (\%)')
plt.xlabel('EOF Mode')
plt.title('Explained Variance of first {} EOFs'.format(m));
plt.legend();

### Plot dominant modes of variability: EOF spatial modes and PC series
The plots show dominant spatial and temporal modes of variability for the normalized dataset. It is difficult to interpret these physically.
#### TODO: Define vmax, vmin params for eofs.plot() if plotting for original cleaned dataset (un-normalized data)

In [ ]:
nmodes_plot = 3 # Number of modes to plot

sns.set_theme(style="white")
fig=plt.figure(figsize=(15,8))
gs = GridSpec(nmodes_plot,2,width_ratios=[1.5,2.5])
ax0 = [fig.add_subplot(gs[i, 0],projection=crs) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0, a1) in enumerate(zip(ax0, ax1)):
    norm_eofs.sel(mode=i+1).plot(ax=a0,
                            cmap='cmo.balance',
                            add_colorbar=True,
                            cbar_kwargs={'orientation': 'vertical',
                                         'label': ''})
    a0.coastlines(resolution='10m',linewidth=0.25,color='0.8')
    a1.plot(norm_pcs.sel(mode=i+1),linewidth=1,color='dimgray')
    a1.set_xlabel('')

a1.set_xlabel('Time (months)');
#cb = plt.colorbar(eofs.sel(mode=1), ax=a0, orientation="vertical", pad=0.15)


"""
# Figure: Dominant EOF modes
sns.set_theme(style="white")
fg = eofs[:,:,0:2].plot(aspect=2,
                        size=8,
                        col="mode", 
                        col_wrap=2,
                        cmap='cmo.balance', # Uses cmocean colorbar - should be installed
                        cbar_kwargs={'orientation': 'vertical',
                                    'label': 'Standard Deviation Units'},
                        subplot_kws=dict(projection=ccrs.SouthPolarStereo()))

[ax.coastlines() for ax in fg.axs.flatten()];
""";

## Fourier Phase Randomization

Now, the projection co-efficients from the previous step are phase-randomized to generate new realizations of the data. In this step, we sample phases for each FFT(PC) from (0,$2\pi$), and iterate to generate n realizations.

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 5
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
nmodes_plot = 4 # Number of modes to plot
nrealizations_to_plot = 5 # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,20))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(new_fl[n_realization,:,i],color='b', linewidth=0.5)
        a1.psd(new_fl[n_realization,:,i],color='b', linewidth=0.5)
    a0.plot(new_fl[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(new_fl[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a0.plot(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a1.psd(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

#### Randomize the Eigen-value scaled PCs (pcs(1))

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = norm_pcs[:,m] # fluxpcs[:,i] when using PCA outputs # pcs[:,m]
        fl_fourier = np.fft.rfft(fl)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl_scaled = np.fft.irfft(fl_fourier_new)
        #scale_fac = np.sqrt(norm_varexpl_values[m]).values
        new_fl[i,:,m] = new_fl_scaled
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
nmodes_plot = 4 # Number of modes to plot
nrealizations_to_plot = 2 # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,20))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(new_fl[n_realization,:,i],color='b', linewidth=0.5)
        a1.psd(new_fl[n_realization,:,i],color='b', linewidth=0.5)
    a0.plot(new_fl[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(new_fl[0,:,i],color='b', noverlap=120,NFFT=256,linewidth=0.25,label='Randomized')
    a0.plot(norm_pcs_eig[:,i],color='k', linewidth=2.5,label='Original')
    a1.psd(norm_pcs_eig[:,i],color='k', noverlap=120,NFFT=256,linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

## Statistical Generation

This uses a custom function presently committed to the `xeofs` package. There is the possibility that this might break should `xeofs` be upgraded.

*Ref: For commit diff details, see [mshiv/xeofs-rand](https://github.com/nicrie/xeofs/compare/main...mshiv:xeofs-rand:main).*
#### TODO: Rewrite generator function using PCA as a standalone package and remove `xeofs` dependency.

In [ ]:
# Generate dataset realizations
# Note standardized input data to the initial PCA 
for i in range(n_realizations):
    flux_reconstr = generate_data(i, 1500, 1)
    flux_reconstr = (flux_reconstr*flux_clean_tstd)+flux_clean_tmean
    flux_reconstr.to_netcdf(inDirName+interim_data_folder+
                            cise_file_path+std_file_path+"norm_scaled-pcs_gen_{}.nc".format(i))
    del flux_reconstr
    gc.collect()
    print('reconstructed realization # {}'.format(i))

## Helper Functions

Functions defined for the generation and follow on calculations

In [ ]:
#modified to not return f - in calculation of RMSE, only Px required
def psd_calc_grid(data,y,x):
    f, Px = scipy.signal.welch(data[:,y,x])
    return Px

def time_series(clipped_data):
    clipped_ts = clipped_data.sum(['y','x'])
    return clipped_ts

# Reconstruct flux dataset using phase randomized PCs
# This section is to be called iteratively for ensemble runs with multiple realizations
# This method also takes 'modes' as a parameter: 
# which is used to reconstruct dataset with different number of selected modes

def generate_data(n_realization,mode,mode_skip):
    flux_reconstr = norm_model.reconstruct_randomized_X(new_fl[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr

def clip_data(total_data, basin):
    clipped_data = total_data.rio.clip(icems.loc[[basin],'geometry'].apply(mapping))
    #clipped_data = clipped_data.dropna('time',how='all')
    #clipped_data = clipped_data.dropna('y',how='all')
    #clipped_data = clipped_data.dropna('x',how='all')
    clipped_data = clipped_data.drop("month")
    return clipped_data

In [ ]:
modes_used = [50, 100, 200, 500, 1500]

for mode_number in modes_used:
    for i in range(n_realizations):
        flux_reconstr = generate_data(i, mode_number, 1)
        flux_reconstr = (flux_reconstr*flux_clean_tstd)+flux_clean_tmean
        flux_reconstr.to_netcdf(inDirName+processed_data_folder+pca_3+"REC_{}-modes_{}.nc".format(modes_used,i))
        del flux_reconstr
        gc.collect()
        print('reconstructed realization # {} for mode case: {}'.format(i,modes_used))

## Input generated dataset files

Versions of these files from different experimental runs are currently housed in the project repo at ```/data/interim```. Note that the ```/data``` directory is ignored for remote commits, and is currently available only on local.
Location can be changed above when saving the file to NETCDF in the Generation step.

In [ ]:
orig = clip_data(flux_clean, basin)
rec0 = clip_data(flux_reconstr0, basin)
rec1 = clip_data(flux_reconstr1, basin)
rec2 = clip_data(flux_reconstr2, basin)

In [ ]:
orig = orig.rename('orig')
rec0 = rec0.rename('rec0')
rec1 = rec0.rename('rec1')
rec2 = rec0.rename('rec2')

combined = xr.merge([orig,rec0,rec1,rec2])

In [ ]:
icems.

In [ ]:
mode_number = 1500
flux_reconstr0 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_0.nc'.format(mode_number))
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__
flux_reconstr1 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_1.nc'.format(mode_number))
flux_reconstr1 = flux_reconstr1.__xarray_dataarray_variable__
flux_reconstr2 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_2.nc'.format(mode_number))
flux_reconstr2 = flux_reconstr2.__xarray_dataarray_variable__

In [ ]:
flux_

In [ ]:
modes_used = [200, 500, 1500]
#catchments = np.array([1,27,28,61,70,83])
catchments = np.array([1,27,28,61])
basins = catchments+33
n_basins = len(basins)
color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for mode_number in modes_used:
    flux_reconstr0 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_0.nc'.format(mode_number))
    flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__
    flux_reconstr1 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_1.nc'.format(mode_number))
    flux_reconstr1 = flux_reconstr1.__xarray_dataarray_variable__
    flux_reconstr2 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_2.nc'.format(mode_number))
    flux_reconstr2 = flux_reconstr2.__xarray_dataarray_variable__
    #basin_combined_datasets = xr.open_mfdataset(inDirName+processed_data_folder+pca_3+"*_{}-modes_{}.nc".format(mode_number,basin),combine="nested",compat="override")
    #basin_combined_datasets.to_netcdf(inDirName+processed_data_folder+pca_3+"combined_{}-modes_{}.nc".format(mode_number,basin))
    #combined = xr.merge([orig,rec0,rec1,rec2])
    #combined.to_netcdf(inDirName+)
    for i,(a0,a1) in enumerate(zip(ax0,ax1)):
        orig = clip_data(flux_clean, basins[i])
        rec0 = clip_data(flux_reconstr0, basins[i])
        rec1 = clip_data(flux_reconstr1, basins[i])
        rec2 = clip_data(flux_reconstr2, basins[i])
        orig = orig.rename('orig')#.to_netcdf(inDirName+processed_data_folder+pca_3+"orig_{}-modes_{}.nc".format(mode_number,basin))
        rec0 = rec0.rename('rec0')#.to_netcdf(inDirName+processed_data_folder+pca_3+"rec0_{}-modes_{}.nc".format(mode_number,basin))
        rec1 = rec1.rename('rec1')#.to_netcdf(inDirName+processed_data_folder+pca_3+"rec1_{}-modes_{}.nc".format(mode_number,basin))
        rec2 = rec2.rename('rec2')#.to_netcdf(inDirName+processed_data_folder+pca_3+"rec2_{}-modes_{}.nc".format(mode_number,basin))
        combined = xr.merge([orig,rec0,rec1,rec2])
        combined_ts = time_series(combined)
        a0.plot(combined_ts.orig,color=color_orig, linewidth=1.5,label='Original')
        a0.plot(combined_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
        a0.plot(combined_ts.rec1,color=color_gen, linewidth=0.25)
        a0.plot(combined_ts.rec2,color=color_gen, linewidth=0.25)
        a1.psd(combined_ts.orig,color=color_orig, linewidth=1.5,label='Original')
        a1.psd(combined_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
        a1.psd(combined_ts.rec1,color=color_gen, linewidth=0.25)
        a1.psd(combined_ts.rec2,color=color_gen, linewidth=0.25)
        a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
        a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
        a0.set_xlabel('')
        a1.set_xlabel('')
        axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                           axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                           axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
        icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
        axins.coastlines(resolution='10m',linewidth=0.25)
        icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='k',linewidth=0.4)
        del orig, rec0, rec1, rec2, combined
    a1.legend();
    a0.set_xlabel('Time (months)');
    a1.set_xlabel('Frequency');
    plt.savefig(inDirName+processed_data_folder+pca_3+'generator-compare_{}-modes'.format(mode_number))
    del flux_reconstr0, flux_reconstr1, flux_reconstr2
    gc.collect()

In [ ]:
flux_reconstr0_ts = time_series(flux_reconstr0)
flux_reconstr1_ts = time_series(flux_reconstr1)
flux_reconstr2_ts = time_series(flux_reconstr2)

In [ ]:
for basin in basins:
    orig = clip_data(flux_clean, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    rec2 = clip_data(flux_reconstr2, basin)
    orig.rename('orig'.format(basin)).to_netcdf(inDirName+processed_data_folder+pca_3+"orig_{}-modes_{}.nc".format(mode_number,basin))
    rec0.rename('rec0'.format(basin)).to_netcdf(inDirName+processed_data_folder+pca_3+"rec0_{}-modes_{}.nc".format(mode_number,basin))
    rec1.rename('rec1'.format(basin)).to_netcdf(inDirName+processed_data_folder+pca_3+"rec1_{}-modes_{}.nc".format(mode_number,basin))
    rec2.rename('rec1'.format(basin)).to_netcdf(inDirName+processed_data_folder+pca_3+"rec2_{}-modes_{}.nc".format(mode_number,basin))
    basin_combined_datasets = xr.open_mfdataset(inDirName+processed_data_folder+pca_3+"*_{}-modes_{}.nc".format(mode_number,basin))
    basin_combined_datasets.to_netcdf(inDirName+processed_data_folder+pca_3+"combined_{}-modes_{}.nc".format(mode_number,basin))
    del orig, rec0, rec1, rec2, basin_combined_datasets
    gc.collect()

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.psd(flux_clean_ts, color='k', linewidth=2.0,label='Model Data')
plt.psd(flux_reconstr0_ts, color='b', linewidth=0.5,label='Generated Data');
plt.psd(flux_reconstr1_ts, color='b', linewidth=0.5);
plt.psd(flux_reconstr2_ts, color='b', linewidth=0.5);

In [ ]:
n_basins = 1 #len(basins)
color_orig = 'k'
color_gen = 'dimgray'
sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5))
plt.suptitle('Full AIS Domain')

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    a0.plot(flux_clean_ts,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(flux_reconstr0_ts,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(flux_reconstr1_ts,color=color_gen, linewidth=0.25)
    a0.plot(flux_reconstr2_ts,color=color_gen, linewidth=0.25)
    a1.psd(flux_clean_ts,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(flux_reconstr0_ts,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(flux_reconstr1_ts,color=color_gen, linewidth=0.25)
    a1.psd(flux_reconstr2_ts,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series')
    a1.set_title('PSD')
    a0.set_xlabel('')
    a1.set_xlabel('')
    axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.plot(ax=axins,color='k',linewidth=0.4)
    #axins.set_axis_off();
#a0.legend();
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#gs.tight_layout(fig)

In [ ]:
# change file name for other realizations
flux_reconstr0 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'norm_scaled-pcs_gen_0.nc')
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__

flux_reconstr1 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'norm_scaled-pcs_gen_1.nc')
flux_reconstr1 = flux_reconstr1.__xarray_dataarray_variable__

In [ ]:

flux_reconstr2 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'norm_flux_gen_2.nc')
flux_reconstr2 = flux_reconstr2.__xarray_dataarray_variable__

flux_reconstr3 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'norm_flux_gen_3.nc')
flux_reconstr3 = flux_reconstr3.__xarray_dataarray_variable__

flux_reconstr4 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'norm_flux_gen_4.nc')
flux_reconstr4 = flux_reconstr4.__xarray_dataarray_variable__

In [ ]:
# Specify catchments to clip data and plot for comparisons
#Old reconstructions: 
# basins = np.array((34,55,114,103,104,105))
# From previous section on chosen catchments (Scatter plot):
# catchments = np.array([1,27,28,61,70,83,94,71,22])
catchments = np.array([1,27,28,61])
basins = catchments+33

# Sample ice shelves chosen:
# Amery = 1 ~ 34
# George VI = 27 ~ 
# Getz = 28
# Pine Island = 61
# Ronne = 70
# Totten = 83
# Wilkins = 94
# Western Ross = 71
# Filchner = 22
# Thwaites = 81
# Eastern_Ross = 72

Create subsets of data (original and reconstructed realizations) for individual catchments. The data variables in each are named so that they may be easily input using the `xarray.open_mfdataset` method. Note however that `CDO` terminal commands are better suited for this if the number of realizations is significantly high (beyond 30?)

In [ ]:
"""
for basin in basins:
    orig = clip_data(flux_clean, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    rec2 = clip_data(flux_reconstr2, basin)
    rec3 = clip_data(flux_reconstr3, basin)
    rec4 = clip_data(flux_reconstr4, basin)
    orig.rename('orig_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"orig_{}.nc".format(basin))
    rec0.rename('rec0_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec0_{}.nc".format(basin))
    rec1.rename('rec1_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec1_{}.nc".format(basin))
    rec2.rename('rec2_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec2_{}.nc".format(basin))
    rec3.rename('rec3_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec3_{}.nc".format(basin))
    rec4.rename('rec4_{}'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec4_{}.nc".format(basin))
    del orig, rec0, rec1, rec2, rec3, rec4
    gc.collect()
"""

#### Clip data to each basin

In [ ]:
for basin in basins:
    orig = clip_data(flux_clean, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    orig.rename('orig'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"orig_{}.nc".format(basin))
    rec0.rename('rec0'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec0_{}.nc".format(basin))
    rec1.rename('rec1'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec1_{}.nc".format(basin))
    del orig, rec0, rec1
    gc.collect()

In [ ]:
for basin in basins:
    orig = clip_data(flux_clean, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    rec2 = clip_data(flux_reconstr2, basin)
    rec3 = clip_data(flux_reconstr3, basin)
    rec4 = clip_data(flux_reconstr4, basin)
    orig.rename('orig'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"orig_{}.nc".format(basin))
    rec0.rename('rec0'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec0_{}.nc".format(basin))
    rec1.rename('rec1'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec1_{}.nc".format(basin))
    rec2.rename('rec2'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec2_{}.nc".format(basin))
    rec3.rename('rec3'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec3_{}.nc".format(basin))
    rec4.rename('rec4'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+"rec4_{}.nc".format(basin))
    del orig, rec0, rec1, rec2, rec3, rec4
    gc.collect()

#### Combiine realizations and original data as separate data variables in `NETCDF` files.

In [ ]:
for basin in basins:
    basin_combined_datasets = xr.open_mfdataset(inDirName+interim_data_folder+
                                                cise_file_path+std_file_path+"*_{}.nc".format(basin),
                                                combine="nested")
    basin_combined_datasets.to_netcdf(inDirName+interim_data_folder+
                                      cise_file_path+std_file_path+"combined_{}.nc".format(basin))
    del basin_combined_datasets
    gc.collect()

#### Plot figure (time series and PSD comparisons for each basin)

In [ ]:
n_basins = len(basins)

color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    combined_ds = xr.open_dataset(inDirName+interim_data_folder+
                              cise_file_path+std_file_path+"combined_{}.nc".format(basins[i]))
    combined_ds_ts = time_series(combined_ds)
    a0.plot(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
    a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
    a0.set_xlabel('')
    a1.set_xlabel('')
    axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='k',linewidth=0.4)
    #axins.set_axis_off();
#a0.legend();
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#gs.tight_layout(fig)

In [ ]:
n_basins = len(basins)

color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    combined_ds = xr.open_dataset(inDirName+interim_data_folder+
                              cise_file_path+std_file_path+"scaled_data_eig/combined_{}.nc".format(basins[i]))
    combined_ds_ts = time_series(combined_ds)
    a0.plot(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
    a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
    a0.set_xlabel('')
    a1.set_xlabel('')
    axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='k',linewidth=0.4)
    #axins.set_axis_off();
#a0.legend();
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#gs.tight_layout(fig)

In [ ]:
n_basins = len(basins)

color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    combined_ds = xr.open_dataset(inDirName+interim_data_folder+
                              cise_file_path+std_file_path+"combined_{}.nc".format(basins[i]))
    combined_ds_ts = time_series(combined_ds)
    a0.plot(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a0.plot(combined_ds_ts.rec2,color=color_gen, linewidth=0.25)
    a0.plot(combined_ds_ts.rec3,color=color_gen, linewidth=0.25)
    a0.plot(combined_ds_ts.rec4,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.rec2,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.rec3,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.rec4,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
    a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
    a0.set_xlabel('')
    a1.set_xlabel('')
    axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='k',linewidth=0.4)
    #axins.set_axis_off();
#a0.legend();
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#gs.tight_layout(fig)

### Plot comparisons of full AIS time series

#### Input full dataset (if not already done) and sum over entire domain to obtain time series.

In [ ]:
# Full AIS data comparisons
flux_clean_ts = time_series(flux_clean)
flux_reconstr0_ts = time_series(flux_reconstr0)
flux_reconstr1_ts = time_series(flux_reconstr1)
flux_reconstr2_ts = time_series(flux_reconstr2)
flux_reconstr3_ts = time_series(flux_reconstr3)
flux_reconstr4_ts = time_series(flux_reconstr4)

#### Plot figure

#### TODO: Add formatting and animate spatial variability fields for 2 generated realizations and original data. 
This will be added as a supplement.

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.plot(flux_clean_ts, color='k', linewidth=1.5,label='Original')
plt.plot(flux_reconstr0_ts, color='b', linewidth=0.25,label='Randomized')
plt.plot(flux_reconstr1_ts, color='b', linewidth=0.25)
plt.plot(flux_reconstr2_ts, color='b', linewidth=0.25)
plt.plot(flux_reconstr3_ts, color='b', linewidth=0.25)
plt.plot(flux_reconstr4_ts, color='b', linewidth=0.25)

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.plot(flux_clean_ts*sec_per_year/rho_fw, color='k', linewidth=1.5,label='Original')
plt.plot(flux_reconstr0_ts*sec_per_year/rho_fw, color='b', linewidth=0.25,label='Randomized')
plt.plot(flux_reconstr1_ts*sec_per_year/rho_fw, color='b', linewidth=0.25)
plt.plot(flux_reconstr2_ts*sec_per_year/rho_fw, color='b', linewidth=0.25)
plt.plot(flux_reconstr3_ts*sec_per_year/rho_fw, color='b', linewidth=0.25)
plt.plot(flux_reconstr4_ts*sec_per_year/rho_fw, color='b', linewidth=0.25)

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.psd(flux_clean_ts, color='k', linewidth=1.5,label='Original')
plt.psd(flux_reconstr0_ts, color='b', linewidth=0.25,label='Randomized');
plt.psd(flux_reconstr1_ts, color='b', linewidth=0.25);
plt.psd(flux_reconstr2_ts, color='b', linewidth=0.25);
plt.psd(flux_reconstr3_ts, color='b', linewidth=0.25);
plt.psd(flux_reconstr4_ts, color='b', linewidth=0.25);

## PSD calculations for AIS domain

In [ ]:
from scipy import signal
freqs0, psd0 = signal.welch(flux_clean_ts)
freqs1, psd1 = signal.welch(flux_reconstr0_ts)
freqs2, psd2 = signal.welch(flux_reconstr1_ts)
freqs3, psd3 = signal.welch(flux_reconstr2_ts)
freqs4, psd4 = signal.welch(flux_reconstr3_ts)
freqs5, psd5 = signal.welch(flux_reconstr4_ts)

In [ ]:
from scipy import signal
freqs0, psd0 = signal.welch(combined_ds_ts.orig)
freqs1, psd1 = signal.welch(combined_ds_ts.rec0)
freqs2, psd2 = signal.welch(combined_ds_ts.rec1)
freqs3, psd3 = signal.welch(combined_ds_ts.rec2)
freqs4, psd4 = signal.welch(combined_ds_ts.rec3)
freqs5, psd5 = signal.welch(combined_ds_ts.rec4)

## PSD plot for AIS domain

This uses a normalized (semilog?) power in the y-axis, as opposed to [dB/Hz] shown earlier.

#### TODO: Check all PSDs for different catchments

In [ ]:
plt.figure(figsize=(15, 5))
plt.semilogx(freqs0, psd0,color='k', linewidth=1.5,label='Original')
plt.semilogx(freqs1, psd1,color='b', linewidth=0.25,label='Randomized')
plt.semilogx(freqs2, psd2,color='b', linewidth=0.25)
plt.semilogx(freqs3, psd3,color='b', linewidth=0.25)
plt.semilogx(freqs4, psd4,color='b', linewidth=0.25)
plt.semilogx(freqs5, psd5,color='b', linewidth=0.25)
plt.ylabel('Power')
plt.xlabel('Frequency ($month^{-1}$)')

### Spectral Filtering on dataset

### TODO: 
#### 3 tests:
    - [ ] Filter on original data input - easiest?
    - [ ] Filter on projection co-efficients before randomization
    - [ ] Filter on generated projection co-efficients after randomization, before generation
    - [ ] Filter after generation - NO

In [ ]:
fig = plt.figure(figsize=(25,6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(flux_clean_ts[:600])
## Filtering of the time series
fs=30/24/3600 #1 month in Hz (sampling frequency)


nyquist = fs / 2 # 0.5 times the sampling frequency
cutoff=0.15 # fraction of nyquist frequency, here  it is 5 days
print('cutoff= ',1/cutoff*nyquist*24*3600,' days') #cutoff=  4.999999999999999  days
b, a = signal.butter(5, cutoff, btype='lowpass') #low pass filter

dUfilt = signal.filtfilt(b, a, flux_clean_ts)
dUfilt=np.array(dUfilt)
dUfilt=dUfilt.transpose()

plt.plot(dUfilt[:600],'r',linewidth=1)

plt.xlabel('Time in months',fontsize=18)
plt.ylabel('Std Deviation',fontsize=18)


In [ ]:


# ax.set_title('Vertical Component CGPS Data')
#plt.savefig('test.png',dpi=150,bbox_inches='tight')